<a href="https://colab.research.google.com/github/2303A52306/gen-ai/blob/main/genai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('/content/healthcare-dataset-stroke-data (2).csv')
data

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [4]:
data['gender'] = data['gender'].replace({'Female': 1, 'Male': 0, 'Other': 2})
data['ever_married'] = data['ever_married'].replace({'Yes': 0, 'No': 1})
data['work_type'] = data['work_type'].replace({'Private': 0, 'Self-employed': 1, 'Govt_job': 2, 'children': 3, 'Never_worked':4})
data['Residence_type'] = data['Residence_type'].replace({'Urban': 1, 'Rural': 0})
data['bmi'] = data['bmi'].fillna(0)
data['smoking_status'] =data['smoking_status'].replace({'formerly smoked': 0, 'never smoked': 1, 'smokes': 2, 'Unknown': 3})
data


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,0,67.0,0,1,0,0,1,228.69,36.6,0,1
1,51676,1,61.0,0,0,0,1,0,202.21,0.0,1,1
2,31112,0,80.0,0,1,0,0,0,105.92,32.5,1,1
3,60182,1,49.0,0,0,0,0,1,171.23,34.4,2,1
4,1665,1,79.0,1,0,0,1,0,174.12,24.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,1,80.0,1,0,0,0,1,83.75,0.0,1,0
5106,44873,1,81.0,0,0,0,1,1,125.20,40.0,1,0
5107,19723,1,35.0,0,0,0,1,0,82.99,30.6,1,0
5108,37544,0,51.0,0,0,0,0,0,166.29,25.6,0,0


In [5]:
X = data.drop('stroke', axis=1).values
y = data['stroke'].values

# Scale the features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Reshape for LSTM input (samples, timesteps, features)
# The original shape is (5110, 11) - 5110 samples, 11 features
# We want to reshape it to (5110, 1, 11) - 5110 samples, 1 timestep, 11 features
X = X.reshape(X.shape[0], 1, X.shape[1])
print(X.shape)

(5110, 1, 11)


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("/content/healthcare-dataset-stroke-data (2).csv")  # Replace with actual dataset path

# Preprocess data (example: encoding categorical variables)
# Convert 'id' column to numeric, or exclude it if not needed.
df['id'] = pd.to_numeric(df['id'], errors='coerce') # Attempt to convert to numeric, replacing errors with NaN
df = df.drop(columns=['id']) # Remove it

for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].astype('category').cat.codes # Convert object type to category then to numeric code


X = df.drop(columns=["stroke"])  # Assuming 'stroke' is the target column
y = df["stroke"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train.values.reshape(-1, X_train.shape[1], 1), y_train, epochs=10, batch_size=32)

# Evaluate model
accuracy = model.evaluate(X_test.values.reshape(-1, X_test.shape[1], 1), y_test)[1]
print(f"Test Accuracy: {accuracy:.2f}")

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


128/128 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9296 - loss: nan
Epoch 2/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9583 - loss: nan
Epoch 3/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9459 - loss: nan
Epoch 4/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9530 - loss: nan
Epoch 5/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9487 - loss: nan
Epoch 6/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9568 - loss: nan
Epoch 7/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9522 - loss: nan
Epoch 8/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9567 - loss: nan
Epoch 9/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9540 - loss: nan
Epoch 10/10
128/128 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9477 - loss: nan
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9405 - loss: nan
Test Accuracy: 0.94


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
np.random.seed(42)  # Set seed for NumPy
import tensorflow as tf
tf.random.set_seed(42)  # Set seed for TensorFlow


# Data Preprocessing
X = data.drop('stroke', axis=1).values
y = data['stroke'].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X = X.reshape(X.shape[0], X.shape[1])  # Removed LSTM reshaping

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add Noise & Remove Random Features
noise = np.random.normal(0, 0.2, X_train.shape)  # Stronger noise
X_train_noisy = X_train + noise

# Get the indices of the features removed during training
random_features_to_remove = np.random.choice(X_train_noisy.shape[1], 2, replace=False)
X_train_noisy = np.delete(X_train_noisy, random_features_to_remove, axis=1)  # Dropping 2 random features

# Build and train a model without LSTM
model = Sequential()
model.add(Dense(units=4, activation='relu', input_shape=(X_train_noisy.shape[1],)))  # Changed activation function
model.add(Dropout(0.10))  # Very high dropout
model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])  # SGD for slower learning

model.fit(X_train_noisy, y_train, epochs=10, batch_size=512, validation_split=0.2)  # Only 1 epoch

# Evaluate the model
# Remove the same features from X_test that were removed from X_train_noisy
X_test_processed = np.delete(X_test, random_features_to_remove, axis=1)
y_pred = (model.predict(X_test_processed) > 0.5).astype("int32")  # Predicting on processed X_test

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.6047 - loss: 0.6990 - val_accuracy: 0.6161 - val_loss: 0.6644
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6448 - loss: 0.6568 - val_accuracy: 0.6540 - val_loss: 0.6258
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6901 - loss: 0.6164 - val_accuracy: 0.7066 - val_loss: 0.5915
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7201 - loss: 0.5864 - val_accuracy: 0.7518 - val_loss: 0.5606
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7627 - loss: 0.5531 - val_accuracy: 0.7995 - val_loss: 0.5331
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7832 - loss: 0.5301 - val_accuracy: 0.8447 - val_loss: 0.5081
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8222 - loss: 0.5037 - val_accuracy: 0.8716 - val_loss: 0.4858
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8452 - loss: 0.4810 - val_accuracy: 0.8936 - val_loss: 0.4653
Epoch 9/10


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Preprocessing
X = data.drop('stroke', axis=1).values
y = data['stroke'].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Reshape for GRU input (samples, timesteps, features)
# Assuming each sample has 1 timestep and all features
X = X.reshape(X.shape[0], 1, X.shape[1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train the GRU model
model = Sequential()
model.add(GRU(units=4, activation='tanh', return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2]))) # GRU layer
model.add(Dropout(0.99)) # Dropout for regularization
model.add(Dense(units=1, activation='sigmoid')) # Output layer

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=256, validation_split=0.2)

# Evaluate the model
y_pred = (model.predict(X_test) > 0.6).astype("int32")


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.9268 - loss: 1.0111 - val_accuracy: 0.3472 - val_loss: 0.7286
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9343 - loss: 0.8705 - val_accuracy: 0.4853 - val_loss: 0.7017
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9281 - loss: 0.8840 - val_accuracy: 0.6320 - val_loss: 0.6803
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9300 - loss: 0.9196 - val_accuracy: 0.7335 - val_loss: 0.6643
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9271 - loss: 0.8620 - val_accuracy: 0.8227 - val_loss: 0.6445
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9326 - loss: 0.7796 - val_accuracy: 0.8655 - val_loss: 0.6263
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9364 - loss: 0.7551 - val_accuracy: 0.9034 - val_loss: 0.6151
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9375 - loss: 0.7401 - val_accuracy: 0.9132 - val_loss: 0.6

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Data Preprocessing
X = data.drop('stroke', axis=1).values
y = data['stroke'].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Reshape for LSTM input (samples, timesteps, features)
X = X.reshape(X.shape[0], 1, X.shape[1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train the X-LSTM model
model = Sequential()
model.add(LSTM(units=4, activation='tanh', return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))  # X-LSTM layer
model.add(Dropout(0.10))  # Dropout for regularization
model.add(Dense(units=1, activation='sigmoid'))  # Output layer

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=256, validation_split=0.2)

# Evaluate the model
y_pred = (model.predict(X_test) > 0.6).astype("int32")

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 140ms/step - accuracy: 0.3505 - loss: 0.7061 - val_accuracy: 0.5770 - val_loss: 0.6875
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6078 - loss: 0.6849 - val_accuracy: 0.8105 - val_loss: 0.6680
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.8169 - loss: 0.6661 - val_accuracy: 0.9474 - val_loss: 0.6493
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.9239 - loss: 0.6474 - val_accuracy: 0.9548 - val_loss: 0.6306
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9453 - loss: 0.6288 - val_accuracy: 0.9548 - val_loss: 0.6113
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.9500 - loss: 0.6097 - val_accuracy: 0.9548 - val_loss: 0.5911
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9536 - loss: 0.5884 - val_accuracy: 0.9548 - val_loss: 0.5695
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9531 - loss: 0.5662 - val_accuracy: 0.9548 - val_loss: 0